In [5]:
import gymnasium as gym
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

In [1]:
import my_envs

/home/mariddc/miniconda3/envs/prim2/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
env_id = "MiniGrid-DistributionalShiftEnv-v0"

In [6]:
# --- 2. Create and wrap the environment ---
# Wrap to get pixel (image) observations instead of symbolic ones
def make_env(render_mode="rgb_array"):
    env = gym.make(env_id, render_mode=render_mode)
    env = RGBImgPartialObsWrapper(env)  # Get RGB image observations
    env = ImgObsWrapper(env)             # Flatten image into np.array for SB3
    return env

In [7]:
# Vectorize for stable-baselines3 compatibility
vec_env = make_vec_env(make_env, n_envs=4)

In [9]:
# --- 3. Create and train the agent ---
model = PPO("CnnPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10_000)


Using cpu device
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | 0.0179   |
| time/              |          |
|    fps             | 1023     |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 8192     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 141        |
|    ep_rew_mean          | 0.0243     |
| time/                   |            |
|    fps                  | 544        |
|    iterations           | 2          |
|    time_elapsed         | 30         |
|    total_timesteps      | 16384      |
| train/                  |            |
|    approx_kl            | 0.04168526 |
|    clip_fraction        | 0.181      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.93      |
|    explained_variance   | 

In [10]:
# --- 4. Test and render one episode ---
test_env = make_env(render_mode='human')
obs, info = test_env.reset()

for step in range(200):
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = test_env.step(action)
    test_env.render() 
    if terminated or truncated:
        obs, info = test_env.reset()

test_env.close()


/home/mariddc/miniconda3/envs/prim2/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/mariddc/miniconda3/envs/prim2/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
